# A tutoiral on how to create a new package in Julia
“sh>” indicates a shell command line, “julia>” denotes the Julia REPL and “pkg>” denotes the pkg mode, which can be entered by typing “]” in REPL

1. in julia, run  
```julia 
using Pkg
Pkg.generate("PkgName")
```
to generate corresponding folder which will contain an "__src__" (has a "__PkgName.jl__") and a file "Project.toml". 

2. (optional) go back to the shell command line, change folder name to be consistent with the convention "PkgName.jl" by running in shell 
```
mv PkgName PkgName.jl
```

3. go into the folder just generated 
```
cd PkgName.jl 
```
then, enter __pkg__ mode, running 
```julia 
(v1.0) pkg> activate .                # enter virtual environment of package PkgName
(Foo) pkg> add Test                   # Test is the module for unit tests, add all deps here 
```
This updates the “[deps]” section of “Project.toml”. We need the “Test” package to run tests for the new package.

Then, go back to the shell,
```
sh> rm Manifest.toml     # (optional) Manifest.toml is not important for our purpose, can remove it
sh> echo 'julia 1.0' > REQUIRE        # but we need this, a "REQUIRE" file 
sh> mkdir -p test && touch test/runtests.jl  # create "test" folder and "runtests.jl" file 
```

4. fill in src/PkgName.jl

edit “__PkgName.jl/src/PkgName.jl__” to write the actual library code. 

5. creating test

edit “test/runtests.jl” following [the tutorial on Unit Test](https://docs.julialang.org/en/v1/stdlib/Test/).

6. running and testing package locally 
```julia
(v1.0) pkg> dev .
(v1.0) pkg> test PkgName
```
Note that, the current working directory should always be __PkgName.jl__.

An example of __PkgName.jl__
```julia 
module PkgName
using Lib  # all packages used 

using BigLib: thing1, thing2

import Base.show

export MyType, foo  # interfaces users can use 

struct MyType
    x
end
```
More examples of Modules can be found from [Julia Module](https://docs.julialang.org/en/v1/manual/modules/).


An example of "runtests.jl" from [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl/blob/master/test/runtests.jl).

In this file, one needs to include PkgName (the package being tested), Test (package to run tests), and any other packages being used.
```julia
using DifferentialEquations, Test, SafeTestsets 

println("Starting tests")
@time @safetestset "Default Discrete Algorithm" begin include("default_discrete_alg_test.jl") end
@time @safetestset "Default ODE Algorithm" begin include("default_ode_alg_test.jl") end
@time @safetestset "Default Steady State Algorithm" begin include("default_steady_state_alg_test.jl") end
@time @safetestset "Default SDE Algorithm" begin include("default_sde_alg_test.jl") end
@time @safetestset "Default RODE Algorithm" begin include("default_rode_alg_test.jl") end
@time @safetestset "Default DDE Algorithm" begin include("default_dde_alg_test.jl") end
@time @safetestset "Default DAE Algorithm" begin include("default_dae_alg_test.jl") end
# each test here is very concise. 
```
One of the tests "default_discrete_alg_test.jl" looks like this: 
```julia 
# at least, needs to use two packages: PkgName and Test
using DifferentialEquations, Test

prob = DiscreteProblem(zeros(2),(0.0,1.0))
sol = solve(prob)
@test typeof(sol.alg) <: FunctionMap
```
"runtests.jl" and "default_discrete_alg_test.jl" should be put under "test" folder.  